In [18]:
import os 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [102]:
# unpack season data
def unpack_season(year_fp,current):
    gw_data = {}
    players = {}
    
    data = {}
    
    player_id_list_fp = os.path.join(year_fp, 'player_idlist.csv')
    players_raw_fp = os.path.join(year_fp, 'players_raw.csv')
    cleaned_players_fp = os.path.join(year_fp,'cleaned_players.csv')
    gw_data_fp = os.path.join(year_fp, 'gws')
    players_fp = os.path.join(year_fp, 'players')
    
    season = 'season-' + year_fp.split('/')[1][2:].replace('-','') + '.csv'
    season_fp = os.path.join(year_fp, season)
    
    
    player_id_list = pd.read_csv(player_id_list_fp)
    data['player_id_list'] = player_id_list
    
    players_raw = pd.read_csv(players_raw_fp)
    data['players_raw'] = players_raw
    
    cleaned_players = pd.read_csv(cleaned_players_fp, encoding = "ISO-8859-1")
    data['cleaned_players'] = cleaned_players
    
    if year_fp.split('/')[1] != current:
        current_season = pd.read_csv(season_fp)
        data['current_season'] = current_season
        
        for file in os.listdir(gw_data_fp):
            filename = os.fsdecode(file)
            key_val = filename.split('.')[0]
            gw_fp = os.path.join(gw_data_fp,filename)
            try:
                gw_data[key_val] = pd.read_csv(gw_fp, encoding = "ISO-8859-1")
            except:
                continue
                
        data['gw_data'] = gw_data
    else:
        fixtures_fp = os.path.join('data', current, 'fixtures.csv')
        data['fixtures'] = pd.read_csv(fixtures_fp)
        
        teams_fp = os.path.join('data', current, 'teams.csv')
        data['teams'] = teams_fp
                
        
    
    for file in os.listdir(players_fp):
        player_gw_fp = os.path.join(players_fp, file, 'gw.csv')
        player_history_fp = os.path.join(players_fp,file,'history.csv')
        players[file] = {}
        try:
            player_gw = pd.read_csv(player_gw_fp)
            players[file]['player_gw'] = player_gw
        except:
            continue
        try:
            player_history = pd.read_csv(player_history_fp)
            players[file]['players_history'] = player_history 
        except:
            continue
    data['players'] = players
    

    
    
    return data

In [103]:
def get_all_seasons(current):
    seasons = {}
    for file in os.listdir('data'):
        season_fp = os.path.join('data', file)
        seasons[file] = unpack_season(season_fp, current)
            
    return seasons 

In [104]:
seasons = get_all_seasons('2019-20')

In [109]:
seasons['2018-19'].keys()

dict_keys(['player_id_list', 'players_raw', 'cleaned_players', 'current_season', 'gw_data', 'players'])

In [110]:
seasons['2019-20'].keys()

dict_keys(['player_id_list', 'players_raw', 'cleaned_players', 'fixtures', 'teams', 'players'])

In [127]:
seasons['2018-19']['players_raw'].columns

Index(['assists', 'bonus', 'bps', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'clean_sheets', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'creativity', 'dreamteam_count', 'ea_index',
       'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name',
       'form', 'goals_conceded', 'goals_scored', 'ict_index', 'id',
       'in_dreamteam', 'influence', 'loaned_in', 'loaned_out', 'loans_in',
       'loans_out', 'minutes', 'news', 'news_added', 'now_cost', 'own_goals',
       'penalties_missed', 'penalties_saved', 'photo', 'points_per_game',
       'red_cards', 'saves', 'second_name', 'selected_by_percent', 'special',
       'squad_number', 'status', 'team', 'team_code', 'threat', 'total_points',
       'transfers_in', 'transfers_in_event', 'transfers_out',
       'transfers_out_event', 'value_form', 'value_season', 'web_name',
       'yellow_cards'],
      dtype='object')

In [131]:
seasons['2018-19']['players_raw'].loc[:,'first_name':'team_code']

,first_name,form,goals_conceded,goals_scored,ict_index,id,in_dreamteam,influence,loaned_in,loaned_out,...,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code
0,Petr,0.0,9,0,20.4,1,False,205.0,0,0,...,3.4,0,27,Cech,1.1,False,1,a,1,3
1,Bernd,3.2,42,0,80.5,2,False,807.2,0,0,...,3.3,0,105,Leno,4.0,False,19,a,1,3
2,Laurent,2.0,23,3,59.1,3,False,456.4,0,0,...,3.6,0,0,Koscielny,0.9,False,6,a,1,3
3,Héctor,0.0,21,0,73.7,4,False,261.6,0,0,...,3.2,0,0,Bellerín,4.5,False,2,i,1,3
4,Nacho,2.7,24,1,83.4,5,False,413.2,0,0,...,3.5,0,0,Monreal,1.3,False,18,a,1,3
5,Rob,0.0,10,0,23.2,6,False,187.8,0,0,...,2.4,0,0,Holding,0.4,False,16,i,1,3
6,Shkodran,1.8,40,2,107.8,7,False,718.6,0,0,...,2.6,0,0,Mustafi,1.3,False,20,a,1,3
7,Sead,0.7,29,0,101.0,8,False,323.2,0,0,...,3.4,0,0,Kolasinac,5.4,False,31,a,1,3
8,Konstantinos,0.3,1,0,5.2,10,False,27.0,0,0,...,0.8,0,0,Mavropanos,0.1,False,27,a,1,3
9,Stephan,0.5,18,0,20.0,11,False,113.0,0,0,...,1.0,0,0,Lichtsteiner,0.8,False,12,a,1,3
